In [0]:
!pip install contractions
!pip install ipdb

  Stored in directory: /root/.cache/pip/wheels/59/24/91/695211bd228d40fb22dff0ce3f05ba41ab724ab771736233f3
Successfully built ipdb


In [0]:
  # this shell script will download and install pytorch and cuda
#!git clone https://gist.github.com/f7b7c7758a46da49f84bc68b47997d69.git
#!sh f7b7c7758a46da49f84bc68b47997d69/pytorch041_cuda92_colab.sh

In [0]:
# !conda remove pytorch torchvision -y
# !pip uninstall torch -y
# !pip uninstall torch -y  # yes twice

# !conda install pytorch torchvision -c pytorch


In [0]:
# !pip3 install torch torchvision

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import contractions
import numpy as np
import string
import re
import os
import nltk
from nltk.corpus import stopwords as sw
# from spellchecker import SpellChecker


In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir('drive/My Drive/')

In [0]:
input = pd.read_csv('train_quora.csv')

In [0]:

input.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306122 entries, 0 to 1306121
Data columns (total 3 columns):
qid              1306122 non-null object
question_text    1306122 non-null object
target           1306122 non-null int64
dtypes: int64(1), object(2)
memory usage: 29.9+ MB


In [0]:
input.shape[0]

1306122

In [0]:
ones_indices= np.array(input[input['target']==1].index)
zeros_indices = np.array(input[input['target']==0].index)
len(ones_indices)

80810

In [0]:
ws=nltk.WordNetLemmatizer()

In [0]:
ws.lemmatize('connecting')

'connecting'

In [0]:
# def undersample(zeros_indices,ones_indices,times):#times denote the normal data = times*fraud data
#     zeros_indices_undersample = np.array(np.random.choice(zeros_indices,(times*len(ones_indices)),replace=False))
#     undersample_data= np.concatenate([ones_indices,zeros_indices_undersample])
#     undersample_data = input.iloc[undersample_data,:].reset_index(drop=True)
#     return(undersample_data)

# df_train=undersample(zeros_indices,ones_indices,1)


NameError: ignored

In [0]:
df_train=input

In [0]:
def cleanData(data):
    cleanedData = "".join([x for x in contractions.fix(re.sub(r'\d+','',data)).lower().strip() if x not in string.punctuation])
    cleanedData = re.findall('\w+',cleanedData)
    cleanedData = " ".join(([ws.lemmatize(x) for x in cleanedData if x not in sw.words('english')]))
    #cleanedData = ([ws.lemmatize(x) for x in cleanedData if x not in sw.words('english')])
    return cleanedData    

In [0]:
vocab2={}
def buildVocab(data):
    textArray = data.apply(lambda x: x.split());
    for inp in textArray:
        for word in inp:
            if word not in vocab:
                vocab[word] = len(vocab)
                vocab2[len(vocab2)]=word
    vocab['unk']=len(vocab);
    vocab2[len(vocab2)]='unk'
    vocab['&pad']=len(vocab);
    vocab2[len(vocab2)]='&pad'
    return vocab;

In [0]:
def prepare_sequence(seq,size):
    features=[]
    idxs = [vocab[w] if w in vocab.keys() else vocab['unk'] for w in seq.split(' ')]
    if len(idxs) < size:
        features = ([vocab['&pad']] * (size - len(idxs))) + idxs
    elif len(idxs) > size:
        features= idxs[:size]
    else:
        features=idxs;
    return features;

In [0]:
def fetchSentence(seq):
  text=''
  for i in seq:
      text=text+vocab2[i]+" ";
  return(text.strip())

In [0]:
df_train['question']=df_train['question_text'].apply(lambda x: cleanData(x))

In [0]:
df_train=df_train[df_train['question'] != ''].reset_index(drop=True)

df_train=df_train[df_train['question'].isna() == False].reset_index(drop=True)

df_train[df_train['question'] == '']

,qid,question_text,target,question


In [0]:
vocab={}
vocab=buildVocab(df_train['question'])

vocab.keys()

dict_keys(['quebec', 'nationalist', 'see', 'province', 'nation', 'adopted', 'dog', 'would', 'encourage', 'people', 'adopt', 'shop', 'velocity', 'affect', 'time', 'space', 'geometry', 'otto', 'von', 'guericke', 'used', 'magdeburg', 'hemisphere', 'convert', 'montra', 'helicon', 'mountain', 'bike', 'changing', 'tyre', 'gaza', 'slowly', 'becoming', 'auschwitz', 'dachau', 'treblinka', 'palestinian', 'quora', 'automatically', 'ban', 'conservative', 'opinion', 'reported', 'liberal', 'view', 'crazy', 'wash', 'wipe', 'grocery', 'germ', 'everywhere', 'thing', 'dressing', 'moderately', 'different', 'modestly', 'ever', 'phase', 'wherein', 'became', 'ignorant', 'loved', 'completely', 'disregarding', 'feelingslives', 'get', 'something', 'go', 'way', 'feel', 'temporarily', 'ease', 'change', 'say', 'feminism', 'calgary', 'flame', 'founded', 'dumbest', 'yet', 'possibly', 'true', 'explanation', 'trump', 'elected', 'use', 'external', 'hard', 'disk', 'o', 'well', 'data', 'storagewill', 'affected', 'living

In [0]:
vocab2[1]

'nationalist'

In [0]:
df_train['len']=df_train['question'].apply(lambda x: len(x.split(' ')))

In [0]:
df_train[df_train['target'] == 1 ][df_train['question'].str.contains("")]['question_text']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


22         Has the United States become the largest dicta...
30         Which babies are more sweeter to their parents...
110        If blacks support school choice and mandatory ...
114        I am gay boy and I love my cousin (boy). He is...
115                     Which races have the smallest penis?
119                        Why do females find penises ugly?
127        How do I marry an American woman for a Green C...
144        Why do Europeans say they're the superior race...
156        Did Julius Caesar bring a tyrannosaurus rex on...
167        In what manner has Republican backing of 'stat...
168        Would Europeans continue to participate in the...
208        Why are Americans, British, Canadians, Austral...
214        If both Honey Singh and Justin Bieber fall fro...
224        Why are liberal minorities so voilent towards ...
235        Can we all now admit that President Trump does...
260        The American economy is growing under Trump’s ...
286        Why don't poo

In [0]:
seq_len=10

In [0]:

df_train['questionEncoded']=df_train['question'].apply(lambda x: prepare_sequence(x,seq_len))

#df_train['questionEncoded'][1]

In [0]:
df_train['questionEncoded'][1]

[213556, 213556, 213556, 5, 6, 7, 8, 9, 10, 11]

In [0]:
idxs = np.arange(len(df_train['questionEncoded']))
def get_trn_val_tst_split(split_frac, idxs):

	np.random.shuffle(idxs)
	train_idxs = idxs[: int(split_frac * len(idxs))]
	rem = idxs[int(split_frac * len(idxs)) : ]
	test_idxs, val_idxs = rem[: 10000], rem[10000 : ]

	return train_idxs, val_idxs, test_idxs

train_idxs, val_idxs, test_idxs = get_trn_val_tst_split(0.8, idxs)

# convert to np array
#labels = np.array(encoded_labels)

get_split = lambda i : (df_train['questionEncoded'][i], df_train['target'][i])
X_train, train_labels = get_split(train_idxs)
X_val, val_labels = get_split(val_idxs)
X_test, test_labels = get_split(test_idxs)
X_common,Labels_common=get_split(np.concatenate((train_idxs,val_idxs)))


In [0]:
np.concatenate((train_idxs,val_idxs))

array([1181993,  325837,  704996, ...,  182671, 1274675, 1206563])

In [0]:
train_labels=train_labels.reset_index(drop=True)
X_train=X_train.reset_index(drop=True)
val_labels=val_labels.reset_index(drop=True)
X_val=X_val.reset_index(drop=True)
test_labels=test_labels.reset_index(drop=True)
X_test=X_test.reset_index(drop=True)

In [0]:
pd.Series(train_labels).value_counts()

0    979885
1     64687
Name: target, dtype: int64

In [0]:
from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0)
os_data_X,os_data_y=os.fit_sample(pd.DataFrame.from_items(zip(X_train.index, X_train.values)).T,train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  after removing the cwd from sys.path.


In [0]:
pd.Series(os_data_y).value_counts()

1    979885
0    979885
dtype: int64

In [0]:
os_data_X[os_data_y == 1][64686].tolist() in X_train.tolist()
#new_texts=[x for x in iter(os_data_X[os_data_y == 1]) if x.tolist() not in X_train.tolist()]
# for i in iter(os_data_X[os_data_y == 1]):
#   if(i.tolist() not in X_train.tolist()):
#     print(fetchSentence(i));
#     break;
print(fetchSentence(os_data_X[os_data_y == 1][64686].tolist()))
print(fetchSentence(os_data_X[os_data_y == 1][64688].tolist()))
print(fetchSentence(os_data_X[os_data_y == 1][64698].tolist()))
print(fetchSentence(os_data_X[os_data_y == 1][64689].tolist()))

&pad &pad many people like feminist know much seems great
&pad &pad &pad &pad &pad &pad &pad space victim first
&pad &pad &pad &pad tough clotting charming pen ethiopia surface
&pad &pad baby cause actually elliptical director circulation often mandatory


In [0]:
sincere_indices = Labels_common[Labels_common == 0].index
len(sincere_indices)
insincere_indices = Labels_common[Labels_common == 1].index
length=len(insincere_indices)
random_indices = np.random.choice(sincere_indices,length, replace=False)
under_sample_indices = np.concatenate([insincere_indices,random_indices])
len(under_sample_indices)
under_sample_labels = Labels_common.loc[under_sample_indices]
under_sample_train_X = X_common.loc[under_sample_indices]
len(under_sample_train_X)

160322

In [0]:
idxs_2 = np.arange(len(under_sample_labels))
under_sample_train_X=under_sample_train_X.reset_index(drop=True)
under_sample_labels=under_sample_labels.reset_index(drop=True)
train_idxs, val_idxs, test_idxs = get_trn_val_tst_split(0.8, idxs_2)

val_idxs=np.concatenate((val_idxs,test_idxs));
get_split = lambda i : (under_sample_train_X[i], under_sample_labels[i])
X_train_oversampled, train_labels_oversampled = get_split(train_idxs)
X_val_oversampled, val_labels_oversampled = get_split(val_idxs)

In [0]:
train_labels_oversampled=train_labels_oversampled.reset_index(drop=True)
X_train_oversampled=X_train_oversampled.reset_index(drop=True)
val_labels_oversampled=val_labels_oversampled.reset_index(drop=True)
X_val_oversampled=X_val_oversampled.reset_index(drop=True)

In [0]:
pd.Series(train_labels_oversampled).value_counts()

1    64271
0    63986
Name: target, dtype: int64

In [0]:
X_train_oversampled[0]

[213556, 213556, 213556, 213556, 213556, 213556, 51, 2469, 100, 264]

In [0]:
def kmax_pooling(x, dim, k):
    index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
    return x.gather(dim, index)

In [0]:
from torch.utils.data import TensorDataset, DataLoader

def get_loader(feats, labels, batch_size):
    data = TensorDataset(torch.as_tensor(feats), torch.as_tensor(labels))
    return DataLoader(data, shuffle=True, batch_size=batch_size)

batch_size = 128
train_loader = get_loader(os_data_X, os_data_y, batch_size)
valid_loader = get_loader(X_val, val_labels, batch_size)
test_loader = get_loader(X_test, test_labels, batch_size)



In [0]:
os_data_X[0]

array([213556, 213556, 213556,    380,   2322,   1809,   2114,   1949,
         1867, 133440])

In [0]:
X_train_oversampled[100]

nan

In [0]:
train_loader_underSampled = get_loader(X_train_oversampled, train_labels_oversampled, batch_size)
valid_loader_underSampled = get_loader(X_val_oversampled, val_labels_oversampled, batch_size)
#test_loader = get_loader(X_test, test_labels, batch_size)

In [0]:
for inp,lab in valid_loader:
  print(inp[0]);
  break

tensor([213556,    115,    147,      7,   2441,    239,    369,   3897,   3253,
           128])


In [0]:
class ncmlmodel(nn.Module):
    def __init__(self,embedding_dim, hidden_dim, vocab_size, target_size,numLayers,dropout):
        super(ncmlmodel,self).__init__();
        self.hidden_dim=hidden_dim;
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim,padding_idx=None)
        self.rnn = nn.LSTM(embedding_dim
                          , hidden_dim
                          ,num_layers=numLayers
                          #,nonlinearity='relu'
                          ,bidirectional=True
                          ,batch_first=True
                          ,bias = True)
        self.dropout = nn.Dropout(dropout)
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        self.hidden_linear=int(hidden_dim/2);
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim,self.hidden_linear),
            nn.BatchNorm1d(self.hidden_linear),
            nn.ReLU(inplace=True),
            nn.Linear(self.hidden_linear,target_size))
        
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        #print(embeds.shape)
        rnn_out, _ = self.rnn(embeds)
        #print(rnn_out.shape) 
        rnn_out = rnn_out.contiguous().view(-1, seq_len, 2, self.hidden_dim)
        #rnn_out_pool=kmax_pooling((rnn_out),2,2)
            # get backward output in first node
        rnn_out_bw = rnn_out[:, 0, 1, :]
            # get forward output in last node
        rnn_out_fw = rnn_out[:, -1, 0, :]
            # we may simple concatenate forward & backward outputs,
            #     or add them, multiply or average; in this case i used average
        rnn_out = torch.add(input=rnn_out_bw, alpha=1, other=rnn_out_fw)
        rnn_out = torch.div(rnn_out, 2)
        #rnn_out_reshaped = rnn_out.contiguous().view(rnn_out.size(0),-1)
        rnn_out = rnn_out.contiguous().view(-1, self.hidden_dim)
        d_out = self.dropout(rnn_out)
        #print(rnn_out_reshaped.shape)
        tag_space = self.fc(d_out)
        #print(tag_space)
        op=torch.sigmoid(tag_space)
        #print(op)
        #sf=torch.softmax(tag_space,1)
        #tag_scores = F.log_softmax(tag_space,dim=1)
        return op

In [0]:

model = ncmlmodel(128, 128, len(vocab), 1,2,0.5)
loss_function = nn.BCELoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=0.001)
# with torch.no_grad():
#         inputs = prepare_sequence(X_train[10].split())
#         scores = model(inputs)
#         print(scores)



In [0]:
model_undersampled = ncmlmodel(128, 128, len(vocab), 1,2,0.5)
loss_function = nn.BCELoss()
optimizer = torch.optim.Adamax(model_undersampled.parameters(), lr=0.001)

In [0]:
from sklearn import metrics
def test(model,data_loader):
    outs=[];
    trueL=[]
    model.eval()
    with torch.no_grad():
        for inputs, labels in data_loader:
            if(train_on_gpu):
              inputs, labels = inputs.cuda(), labels.cuda()
            #inp = torch.as_tensor(i)
            scores = model(inputs)
            if(train_on_gpu):
              scores=scores.cpu()
            tensorSample=scores.data.numpy();
            #print(tensorSample[0].shape)
            flat_list = [int(round(item)) for sublist in tensorSample for item in sublist]
            flat_list2 = [item.item() for item in labels]
            trueL.extend(flat_list2)
            outs.extend(flat_list)
            #print(trueL.shape)
    print("Classification report for classifier %s:\n%s\n" 
      % ('NCML RNN', metrics.classification_report(trueL, outs)))
    print("Confusion matrix:\n%s" % metrics.confusion_matrix(trueL, outs))

In [0]:
def validate(model,data_loader):
    outs=[];
    model.eval()
    val_losses=[]
    with torch.no_grad():
        for inputs, labels in data_loader:
            if(train_on_gpu):
              inputs, labels = inputs.cuda(), labels.cuda()
            #inp = torch.as_tensor(i)
            scores = model(inputs)
            loss = loss_function(scores.squeeze(), labels.float())
            val_losses.append(loss.item())
    print('val loss:{0}'.format(np.mean(val_losses)))

In [0]:
train_on_gpu=torch.cuda.is_available()
if(train_on_gpu):
  model.cuda()
test(model,test_loader)

Classification report for classifier NCML RNN:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      9362
           1       0.06      1.00      0.12       638

    accuracy                           0.06     10000
   macro avg       0.03      0.50      0.06     10000
weighted avg       0.00      0.06      0.01     10000


Confusion matrix:
[[   0 9362]
 [   0  638]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
train_on_gpu=torch.cuda.is_available()
if(train_on_gpu):
  model.cuda()
inpSample=torch.as_tensor(os_data_X[0:2]);
inpSample=inpSample.cuda();
inpSample.view(-1,10).shape
model(inpSample.view(-1,10))

tensor([[0.5129],
        [0.5128]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [0]:
# !pip  pytorchtools

In [0]:
# !pip install pytorchtools
# from pytorchtools import EarlyStopping

In [0]:
import ipdb;
import time

train_on_gpu=torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')
model.train()
epoch=5
#early_stopping = EarlyStopping(patience=patience, verbose=True)
#train_on_gpu=False
if(train_on_gpu):
  model.cuda()
for t in range(epoch):
    model.train()
    start = time.time()
    train_losses=[];
    for inputs, labels in train_loader:
      if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
      model.zero_grad();
      output = model(inputs);
      loss = loss_function(output.squeeze(), labels.float())
      loss.backward()
      train_losses.append(loss.item());
     #nn.utils.clip_grad_norm_(model.parameters(), clip)
      
      optimizer.step()
    end = time.time()
    #print("training time : {0}".format((end-start)/60))
    
    print('epoch:{0} training time - {1}  trainLoss:{2}'.format((t+1),(end-start)/60,np.mean(train_losses)));
    #start = time.time()
    validate(model,valid_loader)
    #end = time.time()
    #print("validation time : {0}".format((end-start)/60))

    
    
test(model,test_loader)

Training on GPU.
epoch:1 training time - 4.785901975631714  trainLoss:0.2704830431049026
val loss:0.2554512407499737
epoch:2 training time - 4.782412377993266  trainLoss:0.20403381517549538
val loss:0.2306022068036799
epoch:3 training time - 4.78452277580897  trainLoss:0.1821910567962674
val loss:0.23248966230659296
epoch:4 training time - 4.777505274613699  trainLoss:0.16660653630989433
val loss:0.21501916457182038
epoch:5 training time - 4.782152195771535  trainLoss:0.15378106680334522
val loss:0.22739416899437573
Classification report for classifier NCML RNN:
              precision    recall  f1-score   support

           0       0.97      0.94      0.95      9362
           1       0.36      0.53      0.43       638

    accuracy                           0.91     10000
   macro avg       0.66      0.73      0.69     10000
weighted avg       0.93      0.91      0.92     10000


Confusion matrix:
[[8759  603]
 [ 299  339]]


In [0]:
text = "";

inputs = torch.as_tensor(prepare_sequence(cleanData(text),seq_len))
print(inputs.view(-1,10))
model.eval();
model.cuda()
inputs=inputs.cuda();
scores = model(inputs.view(-1,10))
scores

tensor([[213556, 213556, 213556, 213556, 213556, 213556, 213556,   2853,     83,
           8875]])


tensor([[0.7972]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [0]:
import ipdb;
import time

train_on_gpu=torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')
model.train()
epoch=100
#train_on_gpu=False
if(train_on_gpu):
  model.cuda()
for t in range(epoch): # again, normally you would NOT do 300 epochs, it is toy data
    # The iteration number in the present epoch
    model.train()
    start = time.time()
    for inputs, labels in train_loader:
      if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
      model.zero_grad();
      output = model(inputs);
      loss = loss_function(output.squeeze(), labels.float())
      loss.backward()
     #nn.utils.clip_grad_norm_(model.parameters(), clip)
      optimizer.step()
    end = time.time()
    #print("training time : {0}".format((end-start)/60))
    
    print('epoch:{0} training time - {1}'.format((t+1),(end-start)/60));
    start = time.time()
    test(model,valid_loader)
    end = time.time()
    print("validation time : {0}".format((end-start)/60))

Training on GPU.
epoch:1 training time - 0.5331619580586752
Classification report for classifier NCML RNN:
              precision    recall  f1-score   support

           0       0.87      0.87      0.87     16224
           1       0.87      0.87      0.87     16092

   micro avg       0.87      0.87      0.87     32316
   macro avg       0.87      0.87      0.87     32316
weighted avg       0.87      0.87      0.87     32316


Confusion matrix:
[[14160  2064]
 [ 2056 14036]]
validation time : 0.04965794483820597
epoch:2 training time - 0.5375058452288309
Classification report for classifier NCML RNN:
              precision    recall  f1-score   support

           0       0.90      0.85      0.87     16224
           1       0.86      0.90      0.88     16092

   micro avg       0.88      0.88      0.88     32316
   macro avg       0.88      0.88      0.88     32316
weighted avg       0.88      0.88      0.88     32316


Confusion matrix:
[[13791  2433]
 [ 1532 14560]]
validation 

KeyboardInterrupt: ignored